In [ ]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [ ]:
db = Database('/home/lessa/smodels-database')

In [3]:
exp = db.getExpResults(analysisIDs=['ATLAS-CONF-2013-007'],dataTypes=['upperLimit'],txnames=['T6ttWWoff'],
                       useSuperseded=True)[0]

In [4]:
tx = exp.getTxNames()[0]
print tx

T6ttWWoff


In [ ]:
data = tx.txnameData._data

In [ ]:
xvals = [pt[0][0][0].asNumber(GeV) for pt in data]
yvals = [pt[0][0][-1].asNumber(GeV) for pt in data]
zvals = [pt[1].asNumber(pb) for pt in data]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xvals, yvals, zvals)
ax.set_xlabel('M1')
ax.set_ylabel('m')
ax.set_zlabel('UL')

plt.show()

In [6]:
m = [[358.*GeV,150.*GeV,75.*GeV],[358.*GeV,150.*GeV,75.*GeV]]
porig = tx.txnameData.flattenMassArray(m)
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
pt = np.dot(pt,tx.txnameData._V)  ## rotate
pt = np.array(pt[:tx.txnameData.dimensionality])
print pt,tx.txnameData.getValueFor(m)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
# ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
    xpts.append(tx.txnameData.tri.points[ipt][0])
    ypts.append(tx.txnameData.tri.points[ipt][1])
    zpts.append(tx.txnameData.xsec[ipt])
    print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

[-165.4629868    0.       ] ret= [ 0.23472]
dp= 1 2 [-165.4629868    0.           0.           0.           0.           0.       ]
2.35E-01 [pb]
[-106.06601718    0.        ] 1.86E-01 
[ -1.06066017e+02   1.00000000e-04] 1.86E-01 
[-176.7766953    0.       ] 2.44E-01 


In [ ]:
print tx.txnameData.countNonZeros(pt),tx.txnameData.dimensionality

In [ ]:
ax.scatter(xpts, ypts, zpts,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('UL')

plt.show()

In [ ]:
uvw = [pt[:2]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e6
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)